# **Approximate Nearest Neighbour Search**

* Local Sensivity Hashing Using FAISS 
* Exhaustive Search using FAISS
* Product Quantization using FAISS
* Trees And Graphs using Annoy
* HNSW using NSMLIB

### **Dynamic score calculation of MF algorithm**
Matrix Factorization (MF) algorithms are becoming more popular these days. In MF, the recommendation score is calculated as the inner product of the generated vector of user items, but if the number of users and items is large, the data size to be retained is very large if the score is pre-calculated for all user x item combinations. It will be.

### **High speed vector calculation library faiss**
Faiss, developed by Facebook Research, is a library that can perform such vector calculations at high speed. It can index a set of vectors (matrix) in the memory in advance and execute score calculation and sorting instantly for the vector given as input.

faiss has the following characteristics:

* GPU and multithreaded support for index operations
* Dimensionality reduction: vectors with large dimensions can be reduced to smaller dimensions using PCA
* Reduction of spatial complexity by dimensional compression
* Quantisation: FAISS emphasises on product quantisation for compressing and storing vectors of large dimensions
* Batch processing i.e searching for multiple queries at a time

In [123]:
# !pip3 install lightfm
# !pip3.6 install annoy
# !pip3 install nmslib

In [120]:
import faiss
import numpy
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from annoy import AnnoyIndex

In [ ]:
# constants
RANDOM_STATE = 0
N_FACTOR = 20
N_RESULT = 10

In [39]:
# load dataset
ratings = numpy.loadtxt(
    'ratings.csv',
    delimiter=',',
    skiprows=1,
    usecols=(0, 1, 2),
    dtype=[('userId', 'i8'), ('movieId', 'i8'), ('rating', 'f8')],
)

# movies = numpy.loadtxt(
#     '../Datasets/movielens-small/movies.csv',
#     delimiter=',',
#     skiprows=1,
#     usecols=(0, 1, 2),
#     dtype=[('movieId', 'i8'), ('title', '|S15'), ('genres', '|S15')],
# )

# movies = numpy.loadtxt('../Datasets/movielens-small/movies.csv',
#    dtype={'names': ('movieId', 'title', 'genre'),
#           'formats': ('i8', '|S15', '|S15')}, delimiter=',', skiprows=1)

In [40]:
users = sorted(numpy.unique(ratings['userId']))
movies = sorted(numpy.unique(ratings['movieId']))

In [41]:
# for later use
user_id2i = {id: i for i, id in enumerate(users)}
movie_id2i = {id: i for i, id in enumerate(movies)}
movie_i2id = {i: id for i, id in enumerate(movies)}

In [42]:
mapped_user = numpy.array(list(map(user_id2i.get, ratings['userId'])))
mapped_movieId = numpy.array(list(map(movie_id2i.get, ratings['movieId'])))
                          
# make sparse matrix
rating_mat = coo_matrix(
    (ratings['rating'], (list(map(user_id2i.get, ratings['userId'])),
                         list(map(movie_id2i.get, ratings['movieId']))))
)

In [43]:
# decompose
model = NMF(n_components=N_FACTOR, init='random', random_state=RANDOM_STATE)
user_mat = model.fit_transform(rating_mat)
movie_mat = model.components_.T

/home/nktech/Desktop/preeti/Fall-2021/CMPE257-Machine-Learning/venv36/lib/python3.6/site-packages/sklearn/decomposition/_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


### **Exhaustive Search**
IndexFlatL2 measures the L2 (or Euclidean) distance between all given points between our query vector, and the vectors loaded into the index. It’s simple, very accurate, but not too fast.

In [88]:
# indexing
movie_index = faiss.IndexFlatL2(N_FACTOR)
movie_index.add(movie_mat.astype('float32'))

In [122]:
"""
Query n vectors of dimension d to the index.
Return at most k vectors. If there are not enough results for a query, the result array is padded with -1s.
"""

def search(user_id, index, n=N_RESULT):
    user_i = user_id2i[user_id]
    user_vec = user_mat[user_i].astype('float32')
    
    scores, indices = index.search(numpy.array([user_vec]), n)
    movie_scores = zip(indices[0], scores[0])

    movies=[
        {
            "id": int(movie_i2id[i]),
            "score": float(s),
        }
        for i, s in movie_scores
    ]
    return movies
    # return [name[i] for i in indices[0]]

In [94]:
search(35, movie_index)

[{'id': 1796, 'score': 0.0030687162652611732},
 {'id': 6185, 'score': 0.0030687162652611732},
 {'id': 1861, 'score': 0.0030687162652611732},
 {'id': 5256, 'score': 0.0030687162652611732},
 {'id': 137, 'score': 0.003425348550081253},
 {'id': 1640, 'score': 0.003425348550081253},
 {'id': 1325, 'score': 0.003425348550081253},
 {'id': 1731, 'score': 0.003425348550081253},
 {'id': 526, 'score': 0.003425348550081253},
 {'id': 3820, 'score': 0.003425348550081253}]

### **Local Sensitivity Hashing**
An algorithmic technique that hashes similar input items into the same "buckets" with high probability.(The number of buckets is much smaller than the universe of possible input items.) Since similar items end up in the same buckets, this technique can be used for data clustering and nearest neighbor search.

In [95]:
# indexing
movie_index_lsh = faiss.IndexLSH(movie_mat.shape[1], 8)
movie_index_lsh.add(movie_mat.astype('float32'))

In [96]:
search(12, movie_index_lsh, 8) 

[{'id': 3547, 'score': 0.0},
 {'id': 3729, 'score': 0.0},
 {'id': 55768, 'score': 0.0},
 {'id': 2553, 'score': 0.0},
 {'id': 2844, 'score': 0.0},
 {'id': 2325, 'score': 0.0},
 {'id': 1180, 'score': 1.0},
 {'id': 1128, 'score': 1.0}]

LightFM is a Python implementation of a number of popular recommendation algorithms for both implicit and explicit feedback, including efficient implementation of BPR and WARP ranking losses. It's easy to use, fast (via multithreaded model estimation), and produces high quality results.

It also makes it possible to incorporate both item and user metadata into the traditional matrix factorization algorithms. It represents each user and item as the sum of the latent representations of their features, thus allowing recommendations to generalise to new items (via item features) and to new users (via user features).

In [99]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens

In [100]:
movielens = fetch_movielens()
train = movielens['train']
test = movielens['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=movielens['item_features'], epochs=20 )

vector = movielens['item_features'] * model.item_embeddings

In [101]:
name = movielens['item_labels']

### **Trees and Graphs - AnnoyIndex**
#### **How does it work**
Using random projections and by building up a tree. At every intermediate node in the tree, a random hyperplane is chosen, which divides the space into two subspaces. This hyperplane is chosen by sampling two points from the subset and taking the hyperplane equidistant from them.

In [102]:
from annoy import AnnoyIndex
x = numpy.array(list(map(numpy.float, ratings['rating'])))
annoy_index = AnnoyIndex(vector.shape[1])

/home/nktech/Desktop/preeti/Fall-2021/CMPE257-Machine-Learning/venv36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
for idx, vec in enumerate(vector):
    annoy_index.add_item(idx, vec.tolist())

search_in_x_trees = annoy_index.build(10)


In [104]:
result = annoy_index.get_nns_by_vector(vector[0].tolist(), 10, search_k=search_in_x_trees)

In [105]:
to_lables = [name[i] for i in result]
to_lables

['Toy Story (1995)',
 'Star Wars (1977)',
 'Return of the Jedi (1983)',
 'Star Trek: First Contact (1996)',
 'Twelve Monkeys (1995)',
 'Mars Attacks! (1996)',
 'Willy Wonka and the Chocolate Factory (1971)',
 'Birdcage, The (1996)',
 'Dragonheart (1996)',
 'Men in Black (1997)']

### **Hierarchical NSW**
Hierarchical NSW incrementally builds a multi-layer structure consisting from hierarchical set of proximity graphs (layers) for nested subsets of the stored elements. The maximum layer in which an element is present is selected randomly with an exponentially decaying probability distribution. This allows producing graphs similar to the previously studied Navigable Small World (NSW) structures while additionally having the links separated by their characteristic distance scales. Starting search from the upper layer together with utilizing the scale separation boosts the performance compared to NSW and allows a logarithmic complexity scaling.
 

In [107]:
import nmslib

In [108]:
nms_index = nmslib.init(method='hnsw', space='cosinesimil')

In [109]:
nms_index.addDataPointBatch(vector)

1682

In [110]:
nms_index.createIndex({'post': 2})

In [111]:
nms_result = nms_index.knnQuery(vector, k=10)

In [112]:
nms_result

(array([1190, 1589, 1617, 1643, 1651, 1661, 1664, 1665, 1666, 1669],
       dtype=int32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [113]:
def to_movie_name(arr):
    return [name[i] for i in arr.tolist()]

to_movie_name(nms_result[0])

['Letter From Death Row, A (1998)',
 'To Have, or Not (1995)',
 'King of New York (1990)',
 'Sudden Manhattan (1996)',
 'Temptress Moon (Feng Yue) (1996)',
 'Rough Magic (1995)',
 "Brother's Kiss, A (1997)",
 'Ripe (1996)',
 'Next Step, The (1995)',
 'Tainted (1998)']

### **Product Quantization**
Product quantization is a great way to compress your large data set vectors to enable faster search results. However, this also affects the accuracy of the results returned. Hence the trade-off between accuracy and memory constraints(or speed) should be critically analyzed while choosing any index in FAISS.

In [114]:
quantizer = faiss.IndexFlatL2(vector.shape[1])

In [115]:
number_of_partition=8
search_in_x_partitions=2
subvector_size=8

prod_quant_index = faiss.IndexIVFPQ(quantizer, vector.shape[1], number_of_partition, search_in_x_partitions, subvector_size)

In [116]:
prod_quant_index.train(vector)

In [117]:
prod_quant_index.add(vector)

In [118]:
result = prod_quant_index.search(vector, 10)[1]
to_movie_name(result[0:1])
# result

[array(['Toy Story (1995)', 'Twelve Monkeys (1995)', 'Phenomenon (1996)',
        'Twister (1996)', 'Willy Wonka and the Chocolate Factory (1971)',
        'Mars Attacks! (1996)', 'Independence Day (ID4) (1996)',
        "Jackie Chan's First Strike (1996)", 'Dragonheart (1996)',
        'Star Trek: First Contact (1996)'], dtype=object)]